In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy import nan
import math
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


import transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy


from sklearn import preprocessing
from sklearn.model_selection import train_test_split


import import_ipynb
from data_loader import Data_Loader
from process_data import propress_data
from process_data import embedding
from model import NERModel

c:\Users\u1158286\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\u1158286\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
class Trainer():
    def __init__(self,
                 train_loader,
                #  dev_loader=None,
                 test_loader,
                 model,
                 output_dir=None,
                 optimizer=Adam(learning_rate=3e-5),
                 epochs=2,
                 device="cpu",
                 best_val_loss = float("inf"),
                 PATIENCE = 3
                ):
        self.output_dir = output_dir
        self.model = model
        self.train_loader = train_loader
        # self.dev_loader = dev_loader
        self.test_loader = test_loader
        self.epochs = epochs
        self.optimizer = optimizer
        self.best_val_loss = best_val_loss
        self.PATIENCE = PATIENCE 
        
     
    def train(self, input_ids, attention_masks, label_ids):
        with tf.GradientTape() as tape:
            logits = self.model((input_ids, attention_masks), training=True)
            seq_lengths = tf.reduce_sum(attention_masks, axis=1)
            loss = self.model.compute_loss(logits, label_ids, seq_lengths)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss
    
    
    
    def val(self, input_ids, attention_masks, label_ids):
        logits = self.model((input_ids, attention_masks), training=False)
        seq_lengths = tf.reduce_sum(attention_masks, axis=1)
        loss = self.model.compute_loss(logits, label_ids, seq_lengths)
        return loss
    
    
    def train_process(self):
        train_dataset = self.train_loader
        val_dataset = self.test_loader
        
        checkpoint = tf.train.Checkpoint(optimizer=self.optimizer, model=self.model)
        checkpoint_manager = tf.train.CheckpointManager(checkpoint, "./checkpoints", max_to_keep=1)
        
        for epoch in range(self.epochs):
            print(f"Epoch {epoch + 1}/{self.epochs}")
            train_loss_avg = tf.keras.metrics.Mean()
            val_loss_avg = tf.keras.metrics.Mean()
            # 训练步骤
            for batch_input_ids, batch_attention_masks, batch_label_ids in train_dataset:
                loss = self.train(batch_input_ids, batch_attention_masks, batch_label_ids)
                train_loss_avg.update_state(loss)
            # 验证步骤
            for val_input_ids_batch, val_attention_masks_batch, val_label_ids_batch in val_dataset:
                val_loss = self.val(val_input_ids_batch, val_attention_masks_batch, val_label_ids_batch)
                val_loss_avg.update_state(val_loss)
            # 获取 epoch 损失
            train_loss = train_loss_avg.result()
            val_loss = val_loss_avg.result()
            print(f"Train Loss: {train_loss.numpy()}, Validation Loss: {val_loss.numpy()}")
            # Early Stopping 检查
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                patience_counter = 0
                checkpoint_manager.save()
                print(f"New best model saved with validation loss: {self.best_val_loss.numpy()}")
            else:
                patience_counter += 1
                print(f"Patience Counter: {patience_counter}")
            if patience_counter >= self.PATIENCE:
                print("Early stopping triggered.")
                break
            print("训练完成！")

In [3]:
def main(data_path):
    df = pd.read_csv(data_path,encoding= 'unicode_escape')
    df = df[:1000]
    X_train,X_test,y_train,y_test = propress_data(df).split_train_test()
    input_ids_train,attention_mask_train, label_ids_train = embedding().tokenize(X_train,y_train)
    input_ids_test,attention_mask_test, label_ids_test = embedding().tokenize(X_test,y_test)
    train_loader, test_loader = Data_Loader().data_loader(input_ids_train,attention_mask_train, label_ids_train,input_ids_test,attention_mask_test, label_ids_test)
    
    bert_model = TFBertModel.from_pretrained(r'C:\Users\u1158286\OneDrive - IQVIA\Desktop\NER\bert_model\bert-base-uncased',from_pt = True)
    ner_model = NERModel(bert_model,16)

    
    Trainer(train_loader,test_loader,ner_model).train_process()
    

In [4]:
if __name__ == "__main__":
    data_path = r'C:\Users\u1158286\OneDrive - IQVIA\Desktop\NER\data\ner_dataset.csv'
    main(data_path)

<string>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 9/9 [00:00<00:00, 90.21it/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model tra

Epoch 1/2
Train Loss: 9.411261558532715, Validation Loss: 10.401260375976562
New best model saved with validation loss: 10.401260375976562
训练完成！
Epoch 2/2
Train Loss: 8.880681991577148, Validation Loss: 10.124919891357422
New best model saved with validation loss: 10.124919891357422
训练完成！
